In [1]:
import copy
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# TODO: 
#  1. investigate living dangle Neurons
#  2. investigate gradient size (bigger than lr? limit to lr)
#  3. Collect stats on gradients, neuron values, over training step
#  4. Fix scrolling
#  5. Use Cuda
#  6a. Relu -> Relu6
#  6b. Improve death, introduce train mode/ death mode (kill any which reach zero)
# After that can think on Cifar10 convnet death

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
from ai_libs.basic.deep_reload import deep_reload
import livenet
deep_reload(livenet)
import ai_libs.simple_log as simple_log
from ai_libs.simple_log import LOG
simple_log.level = simple_log.LogLevel.INFO
import math
from matplotlib import pyplot as plt
plt.ion()
%matplotlib TkAgg
livenet.core.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



2.3.1+cu121


'%.4f'

In [175]:
downscale = (2, 2)
test = livenet.datasets.get_mnist_test()
train = livenet.datasets.get_mnist_train()
test_x, test_y = livenet.datasets.to_plain(*test, downscale=downscale, to_gray=True, to_odd=True)
train_x, train_y = livenet.datasets.to_plain(*train, downscale=downscale, to_gray=True, to_odd=True)


In [178]:
network = livenet.nets.create_perceptron(train_x.shape[1], 12, 2)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)

Iˈ0.000 S0 became useless livenet/core/death.py:68
Iˈ0.000 S1 became useless livenet/core/death.py:68
Iˈ0.000 S2 became useless livenet/core/death.py:68
Iˈ0.001 S3 became useless livenet/core/death.py:68
Iˈ0.001 S4 became useless livenet/core/death.py:68
Iˈ0.001 S5 became useless livenet/core/death.py:68
Iˈ0.001 S6 became useless livenet/core/death.py:68
Iˈ0.001 S7 became useless livenet/core/death.py:68
Iˈ0.001 S8 became useless livenet/core/death.py:68
Iˈ0.001 S9 became useless livenet/core/death.py:68
Iˈ0.001 S10 became useless livenet/core/death.py:68
Iˈ0.001 S11 became useless livenet/core/death.py:68
Iˈ0.001 S12 became useless livenet/core/death.py:68
Iˈ0.001 S13 became useless livenet/core/death.py:68
Iˈ0.002 S14 became useless livenet/core/death.py:68
Iˈ0.002 S15 became useless livenet/core/death.py:68
Iˈ0.002 S16 became useless livenet/core/death.py:68
Iˈ0.002 S17 became useless livenet/core/death.py:68
Iˈ0.002 S18 became useless livenet/core/death.py:68
Iˈ0.002 S19 became use

In [253]:
# simple_log.level = simple_log.LogLevel.DEBUG
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.01
trainer.step(2000)


Iˈ0.000 0.381 = 0.332+0.049reg params=74 livenet/net_trainer.py:93
Iˈ0.019 0.395 = 0.346+0.049reg params=74 livenet/net_trainer.py:93
Iˈ0.038 0.406 = 0.356+0.049reg params=74 livenet/net_trainer.py:93
Iˈ0.056 0.431 = 0.381+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.075 0.433 = 0.383+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.093 0.412 = 0.362+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.111 0.361 = 0.311+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.143 0.399 = 0.349+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.163 0.407 = 0.357+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.182 0.464 = 0.415+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.200 0.417 = 0.368+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.220 0.425 = 0.375+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.240 0.438 = 0.388+0.050reg params=74 livenet/net_trainer.py:93
Iˈ0.259 0.376 = 0.326+0.051reg params=74 livenet/net_trainer.py:93
Iˈ0.277 0.342 = 0.292+0.051reg params=74 livenet/net_trainer.p

In [163]:
trainer.history = []

In [254]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y.numpy() - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.9265'

In [259]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["N3->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

#plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
#plt.grid()
plt.plot(values)
values[15000]
# accum = core.stat_utils.AccumStat()
# accum.add_value(trainer.network.parameters())
# accum.plot()


Using matplotlib backend: tkagg


-3.5513

In [207]:
sum(train_y)

tensor([30508])

In [255]:
print(network.context.health_stat.get_stat())
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][-1]) / 10000):
    print(name, f"{p.item():.2f}")


{'dangle': {'RegularNeuron': 2, 'DestinationNeuron': 0}, 'useless': {'RegularNeuron': 0, 'DataNeuron': 152}}
D0 -1.71
D1 1.72
N0 -1.88
N1 -7.54
N3 -224.33
N4 -13.18
N11 -224.58
N3->D0 -8.69
N0->D1 -1.32
N1->D1 -0.67
N3->D1 12.67
N4->D1 1.95
N11->D0 2.53
N11->D1 -22.13
S35->N0 0.46
S52->N0 -0.37
S54->N0 -1.14
S64->N0 -0.49
S91->N0 -0.51
S95->N0 0.89
S48->N1 -1.97
S49->N1 -0.73
S50->N1 -0.89
S62->N1 -0.28
S63->N1 -4.55
S65->N1 -0.38
S77->N1 0.16
S79->N1 -0.83
S81->N1 -0.96
S89->N1 2.09
S34->N4 -0.17
S62->N4 0.41
S77->N4 1.40
S78->N4 1.03
S106->N0 -0.58
S107->N0 -0.67
S117->N0 0.56
S129->N0 1.92
S130->N0 1.05
S131->N0 1.19
S132->N0 -0.14
S142->N0 0.19
S144->N0 0.07
S145->N0 0.70
S159->N0 0.51
S177->N0 -6.70
S102->N1 1.04
S104->N1 1.42
S106->N1 2.04
S115->N1 4.09
S118->N1 0.67
S119->N1 1.44
S130->N1 -10.14
S132->N1 4.06
S148->N1 1.09
S158->N1 -3.11
S159->N1 -2.58
S163->N1 2.17
S101->N4 -11.69
S106->N4 -0.37
S116->N4 -2.75
S117->N4 -0.97
S118->N4 -0.52
S120->N4 -0.90
S129->N4 3.34
S130->N4 

In [191]:
[x.name for x in network.context.health_stat.dangle]

['N3', 'N11']